In [37]:
from collections import deque
import copy
import random
import time


listeClient = [5, 8, 3, 6, 12]
depot = 0


def voisinMinPoid(matrice, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrice[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrice[cur][i]
        elif matrice[cur][i] > 0 and matrice[cur][i] < poidMinTrajet:
            poidMinTrajet = matrice[cur][i]
            nextVoisin = i

    return nextVoisin

def voisinsClientGrapheMatrice(matrice, sommet):
    voisins = [i for i in listeClient if matrice[sommet][i] > 0]
    return voisins 

def poidCycle(matrice, cycle):
    poids_total = 0
    for i in range(len(cycle) - 1):
        poids_total += matrice[cycle[i]][cycle[i + 1]]
    poids_total += matrice[cycle[-1]][cycle[0]] 
    return poids_total



def recherche_tabou_cycle(matrice, start, firstNeighbor, iter_max=100):

    # On copie la matrice pour ne pas modifier l’originale
    matrice_copy = copy.deepcopy(matrice)

    # Le cycle que nous construisons (liste d’indices de sommets)
    cycle = [start]

    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(listeClient) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)

    # Le sommet courant (celui où on se trouve actuellement)
    cur = start

    # Boucle principale de la recherche tabou
    for _ in range(iter_max):

        if len(cycle) == 1:
            voisin = firstNeighbor
        else:
            voisins = voisinsClientGrapheMatrice(matrice_copy, cur) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin = voisinMinPoid(matrice_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrice_copy[cur][voisin] = 0
        matrice_copy[voisin][cur] = 0
        
        cycle.append(voisin) # On ajoute ce voisin au cycle    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur = voisin

    # On retourne le chemin (cycle) trouvé
    return cycle



def tabou_multi_start(matrice, nb_lancements=10, iter_max=100):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    - iter_max : nombre d’itérations par recherche
    """

    meilleur_cycle = []  # Le meilleur cycle global (le plus court)
    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(nb_lancements):

        start = depot

        firstNeighbor = 0

        while matrice[start][firstNeighbor] == 0:
            firstNeighbor = random.randint(0, len(listeClient)-1)

        # On effectue une recherche tabou locale à partir de ce sommet
        cycle = recherche_tabou_cycle(matrice, start, firstNeighbor, iter_max)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1}: départ={firstNeighbor}, longueur du cycle={len(cycle)}, temps du trajet={poidCycle(matrice, cycle)}")

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle(matrice, cycle)
            meilleur_cycle = cycle
            goodI = i+1
        elif poidCycle(matrice, cycle) < tempsMeilleurCycle:
            meilleur_cycle = cycle
            tempsMeilleurCycle = poidCycle(matrice, cycle)
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return meilleur_cycle, tempsMeilleurCycle, goodI



matrice = [
 [0, 12, 5, 19, 1, 8, 16, 11, 6, 7, 13, 17, 10],
 [12, 0, 4, 3, 15, 18, 20, 8, 19, 11, 14, 5, 9],
 [5, 4, 0, 14, 17, 10, 2, 13, 7, 18, 8, 9, 16],
 [19, 3, 14, 0, 12, 6, 9, 20, 15, 2, 10, 11, 7],
 [1, 15, 17, 12, 0, 13, 11, 3, 8, 4, 16, 14, 5],
 [8, 18, 10, 6, 13, 0, 5, 9, 20, 16, 3, 15, 19],
 [16, 20, 2, 9, 11, 5, 0, 7, 14, 19, 18, 12, 4],
 [11, 8, 13, 20, 3, 9, 7, 0, 10, 17, 2, 5, 15],
 [6, 19, 7, 15, 8, 20, 14, 10, 0, 12, 9, 16, 18],
 [7, 11, 18, 2, 4, 16, 19, 17, 12, 0, 15, 8, 6],
 [13, 14, 8, 10, 16, 3, 18, 2, 9, 15, 0, 19, 20],
 [17, 5, 9, 11, 14, 15, 12, 5, 16, 8, 19, 0, 13],
 [10, 9, 16, 7, 5, 19, 4, 15, 18, 6, 20, 13, 0]
]


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")

# Lancement du multi-start (10 essais, taille tabou = 5, 100 itérations max)
meilleur_cycle, tempsMeilleurCycle, goodI = tabou_multi_start(matrice, nb_lancements=10, iter_max=100)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "Longueur du cycle :", len(meilleur_cycle), "  Temps du cycle :", tempsMeilleurCycle)
for s in meilleur_cycle:
    print(s + 1, "-> ", end='')
print(meilleur_cycle[0]+1)  # on revient au départ pour fermer le cycle

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Lancement 1: départ=3, longueur du cycle=6, temps du trajet=58
Lancement 2: départ=2, longueur du cycle=7, temps du trajet=50
Lancement 3: départ=2, longueur du cycle=7, temps du trajet=50
Lancement 4: départ=1, longueur du cycle=7, temps du trajet=54
Lancement 5: départ=4, longueur du cycle=7, temps du trajet=42
Lancement 6: départ=3, longueur du cycle=6, temps du trajet=58
Lancement 7: départ=1, longueur du cycle=7, temps du trajet=54
Lancement 8: départ=1, longueur du cycle=7, temps du trajet=54
Lancement 9: départ=1, longueur du cycle=7, temps du trajet=54
Lancement 10: départ=3, longueur du cycle=6, temps du trajet=58

=== Meilleur cycle trouvé ===
Lancement n° 5 Longueur du cycle : 7   Temps du cycle : 42
1 -> 5 -> 13 -> 7 -> 6 -> 4 -> 9 -> 1

Temps d'exécution : 2.29 ms
